## Select the analysis mode from the dropdown menu below.

In [3]:
import ipywidgets as widgets
from ipywidgets import interact

def set_analysis_type(analysis_mode):
    global analysis_type
    analysis_type = analysis_mode
    print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2'])

interactive(children=(Dropdown(description='analysis_mode', options=('fake analysis', 'fake analysis 2'), valu…

## Enter the filepath to the run you wish to analyze.

In [4]:
global watchfile
watchfile = input(r'Enter path, e.g. 07\200729\foo\ for run foo on Jul 29, 2020: ') + 'image_log.csv' #r'07\200729\foo\image_log.csv'

Enter path, e.g. 07\200729\foo\ for run foo on Jul 29, 2020: 07\200729\foo\


## Run the cell below to begin analyzing.

Check the output; it will update as images are analyzed.

To stop the analysis, press the interrupt kernel/stop button at the very top. You can restart the analysis by simply running the cell below again. The console output will be cleared, but not to worry, it's all in the log file.

In [11]:
'''
breadboard_python_watchdog.py
=============================
This lets you watch a folder for new single images and upload the metadata to Breadboard. 
Usage:

python3 breadboard_python_watchdog.py [WATCHFOLDER]

where [WATCHFOLDER] is the folder your camera program writes images to.

'''

# Imports
import os
import time
import datetime
import shutil
import posixpath
import sys
# from breadboard import BreadboardClient
import warnings
import pandas as pd
from numpy import isnan
import ipywidgets as widgets
from ipywidgets import interact


# analysis_type = 'fake analysis'

# define analysis_function which accepts image filepath and returns dictionary of {analyzed_var_name:analysis_value}
if analysis_type == 'fake analysis':
    from random import randint
    def analysis_function(filepath):
#         time.sleep(2)
        return {'fake analysis 1': randint(0,42), 'fake analysis 2': randint(0,42)}
    analyzed_var_names = ['fake analysis 1', 'fake analysis 2']
elif analysis_type == 'fake analysis 2':
    from random import randint
    def analysis_function(filepath):
#         time.sleep(2)
        return {'fake analysis 3': randint(0,42), 'fake analysis 4': randint(0,42)}
    analyzed_var_names = ['fake analysis 3', 'fake analysis 4']
else:
    print('loading matlab engine...')
    import matlab.engine
    eng = matlab.engine.start_matlab()
    print('matlab engine loaded')
    from MATLABanalyzer import MATLABanalyzer
    analyzer = MATLABanalyzer(eng)

if analysis_type == 'Ncntbare':
    def analysis_function(filepath):
        return {'bareNcntAverageMarqueeBoxValues': analyzer.get_analysis(filepath)['bareNcntAverageMarqueeBoxValues']}
    analyzed_var_names = ['bareNcntAverageMarqueeBoxValues'] #TODO import var_names from MATLAB function instead of hardcoding
    
warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings(
    "ignore", "Could not find appropriate MS Visual C Runtime")
# bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')

def translate_stringy_list(stringy_list):
    #pd.read_csv imports listboundvariables dtype as a string, we want it in list form
    final_list = []
    stringy_list = stringy_list.split(',')
    for var in stringy_list:
        final_list.append(var.translate({ord(c): None for c in '[ ]\,\''}))
    return final_list

def analysis_to_imagelog(image_log_df, idx, outputlog_filename):
    image_filename = image_log_df.loc[idx, 'filename0']
    analysis_dict = analysis_function(image_filename)
    for key in analysis_dict:
        image_log_df.loc[idx,key] = analysis_dict[key]
    image_log_df.to_csv(outputlog_filename, index = False)
    print('{file} analyzed: '.format(file=image_filename) + str(analysis_dict))

def main():
    # Global settings
    # bc = BreadboardClient(config_path='API_CONFIG.json') # enter your path to the API_config
    refresh_time = 0.5  # seconds
    global watchfile
#     watchfile = r'07\200729\foo\image_log.csv'
    print("\n\n Watching this file for changes: " + watchfile + "\n\n")

#     length_old = 0

    # Main Loop
    while True:
        if not os.path.exists(watchfile):
            time.sleep(0.5)
            continue
        else:
            image_log_df = pd.read_csv(watchfile)

        for var_name in analyzed_var_names:
            if var_name not in image_log_df.columns: #start new analysis type
#             print('{analysis} analysis started'.format(analysis = analysis_type))
                analysis_to_imagelog(image_log_df, 0, watchfile)
                if 'analyzed_variables' not in image_log_df.columns:
                    image_log_df.loc[0,'analyzed_variables'] = str(analyzed_var_names)
                    image_log_df.to_csv(watchfile, index = False)
                else:
                    old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[0,'analyzed_variables']))
                    updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                    image_log_df.loc[0,'analyzed_variables'] = str(updated_analyzed_variables)
                    image_log_df.to_csv(watchfile, index = False)
            else: #queue un-analyzed images
                for idx in reversed(range(len(image_log_df.index))): #start from most recent image first
                    if isnan(image_log_df.loc[idx, var_name]):
                        analysis_to_imagelog(image_log_df, idx, watchfile)
                        if isinstance(image_log_df.loc[idx,'analyzed_variables'],float):
                            image_log_df.loc[idx,'analyzed_variables'] = str(analyzed_var_names)
                            image_log_df.to_csv(watchfile, index = False)
                        else:
                            old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[idx,'analyzed_variables']))
                            updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                            image_log_df.loc[idx,'analyzed_variables'] = str(updated_analyzed_variables)
                            image_log_df.to_csv(watchfile, index = False)

    # check if a new image has come in
#         if len(image_log_df.index) > length_old:
#             image_filename = image_log_df.loc[len(
#                 image_log_df.index) - 1, 'filename']
#             print('New image: ' + image_filename + '. Analyzing now...')
#             analysis_variable = analysis_function(image_filename)
#             image_log_df.loc[len(image_log_df.index) - 1,
#                              analysis_type] = analysis_variable
#             image_log_df.to_csv('image_log.csv', index = False)
#             print('analysis value ' + str(analysis_variable))

            # # Write to Breadboard
            # try:
            #     resp = bc.post_images(
            #                 image_names = os.path.splitext(output_filename)[0],
            #                 filepath = output_filepath,
            #                 image_times = [dt],
            #                 auto_time = False # Add more information here
            #                 )
            #     if resp.status_code!=200:
            #         print(resp.text)
            # except: pass
            # names_old = names
#             length_old = len(image_log_df.index)

        # Wait before checking again
        time.sleep(refresh_time)

main()



 Watching this file for changes: 07\200729\foo\image_log.csv


image00.csv analyzed: {'fake analysis 1': 16, 'fake analysis 2': 3}
image40_165242.csv analyzed: {'fake analysis 1': 6, 'fake analysis 2': 16}
image30_165236.csv analyzed: {'fake analysis 1': 28, 'fake analysis 2': 27}
image20_165230.csv analyzed: {'fake analysis 1': 20, 'fake analysis 2': 38}
image10_165224.csv analyzed: {'fake analysis 1': 26, 'fake analysis 2': 26}
image00_165218.csv analyzed: {'fake analysis 1': 8, 'fake analysis 2': 2}
image40.csv analyzed: {'fake analysis 1': 25, 'fake analysis 2': 30}
image30.csv analyzed: {'fake analysis 1': 36, 'fake analysis 2': 38}
image20.csv analyzed: {'fake analysis 1': 1, 'fake analysis 2': 24}
image10.csv analyzed: {'fake analysis 1': 11, 'fake analysis 2': 37}


KeyboardInterrupt: 